In [2]:
import pandas as pd
df = pd.read_csv('./data.csv')

In [3]:
for name in df.columns:
    print(f'- {name} -------------------------')
    print(df[name].value_counts())

- ID -------------------------
1     1
2     1
21    1
20    1
19    1
18    1
17    1
16    1
15    1
14    1
13    1
12    1
11    1
10    1
9     1
8     1
7     1
6     1
5     1
4     1
3     1
22    1
Name: ID, dtype: int64
- name -------------------------
Alice        1
Bob          1
Valentino    1
Urine        1
Tate         1
Stewart      1
Ralf         1
Paton        1
ONeill       1
Nathan       1
Marries      1
Louis        1
Kate         1
Jamie        1
Iris         1
Helen        1
Gordon       1
Fouler       1
Eli          1
Derrick      1
Charles      1
Warner       1
Name: name, dtype: int64
- SwE -------------------------
1    13
0     9
Name: SwE, dtype: int64
- PdM -------------------------
0    19
1     3
Name: PdM, dtype: int64
- Biz -------------------------
0    19
1     3
Name: Biz, dtype: int64
- CS -------------------------
0    19
1     3
Name: CS, dtype: int64


In [4]:
len_members = len(df)
len_members

22

In [5]:
len_groups = int(len_members / 3)
len_groups

7

## Requirements
- Members must be assigned to one group
- Group members must be 3 to 4
- Max members of SwE must be 2

In [53]:
import pulp
problem = pulp.LpProblem("Group", pulp.LpMaximize)

In [54]:
## Variables

In [55]:
import string
G = list(string.ascii_uppercase[:len_groups])
G

['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [56]:
M = df['ID'].tolist()
M

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

In [57]:
MG = [(m,g) for m in M for g in G]
pd.Series(MG)

0       (1, A)
1       (1, B)
2       (1, C)
3       (1, D)
4       (1, E)
        ...   
149    (22, C)
150    (22, D)
151    (22, E)
152    (22, F)
153    (22, G)
Length: 154, dtype: object

In [58]:
X = pulp.LpVariable.dicts('X', MG, cat=pulp.LpBinary)

## Constraints

In [59]:
# Members must be assigned to one group
for m in M:
    problem += pulp.lpSum(X[m,g] for g in G) == 1

In [60]:
# Group members must be 3 to 4
for g in G:
    problem += pulp.lpSum(X[m,g] for m in M) >= 3
    problem += pulp.lpSum(X[m,g] for m in M) <= 4

In [61]:
# Max members of SwE must be 2
M_swe = df[df['SwE'] == 1]['ID'].tolist()
M_swe

[1, 2, 3, 4, 6, 7, 9, 12, 15, 16, 18, 19, 21]

In [62]:
for g in G:
    problem += pulp.lpSum(X[m,g] for m in M_swe) <= 2

In [63]:
status = problem.solve()
print(pulp.LpStatus[status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ta.nakamura/src/github.com/na9amura/try-or-tools/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/70f10899d39544128c5567399b413df6-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/70f10899d39544128c5567399b413df6-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 48 COLUMNS
At line 911 RHS
At line 955 BOUNDS
At line 1111 ENDATA
Problem MODEL has 43 rows, 155 columns and 553 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0005I 22 SOS with 154 members
Cgl0004I processed model has 36 rows, 154 columns (154 integer (154 of which binary)) and 399 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of

In [67]:
for g in G:
    print(f'- Group: {g} --------------')
    for m in M:
        print(X[m,g].value())

- Group: A --------------
0.0
0.0
0.0
0.0
1.0
0.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
- Group: B --------------
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
- Group: C --------------
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
- Group: D --------------
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
- Group: E --------------
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
- Group: F --------------
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
- Group: G --------------
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [68]:
result_df = df.copy()

In [69]:
result_df['group'] = result_df['ID'].map({ m:g for m in M for g in G if X[m,g].value() == 1})

In [70]:
result_df

,ID,name,SwE,PdM,Biz,CS,group
0,1,Alice,1,0,0,0,G
1,2,Bob,1,0,0,0,D
2,3,Charles,1,0,0,0,C
3,4,Derrick,1,0,0,0,B
4,5,Eli,0,0,0,1,A
5,6,Fouler,1,0,0,0,B
6,7,Gordon,1,0,0,0,A
7,8,Helen,0,0,1,0,A
8,9,Iris,1,0,0,0,A
9,10,Jamie,0,1,0,0,G


In [71]:
result_df.sort_values('group')

,ID,name,SwE,PdM,Biz,CS,group
4,5,Eli,0,0,0,1,A
6,7,Gordon,1,0,0,0,A
7,8,Helen,0,0,1,0,A
8,9,Iris,1,0,0,0,A
21,22,Warner,0,0,1,0,B
3,4,Derrick,1,0,0,0,B
5,6,Fouler,1,0,0,0,B
12,13,Marries,0,0,0,1,C
2,3,Charles,1,0,0,0,C
20,21,Valentino,1,0,0,0,C
